In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
%matplotlib inline

In [ ]:
lectures = pd.read_csv('../input/riiid-test-answer-prediction/lectures.csv')
ex_submission = pd.read_csv('../input/riiid-test-answer-prediction/example_sample_submission.csv')
ex_test = pd.read_csv('../input/riiid-test-answer-prediction/example_test.csv')
questions = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv')
train = pd.read_csv('../input/riiid-test-answer-prediction/train.csv', nrows = 99999)

In [ ]:
train.head()

Remove the content_type_id is 1

In [ ]:
train = train.drop(train[train['content_type_id']==1].index)

In [ ]:
train

Change name content_id into question_id, so we can merge with questions dataframe

In [ ]:
train = train.rename(columns={'content_id':'question_id'})

In [ ]:
train

Merge train and questions dataframe

In [ ]:
train = train.merge(questions)

In [ ]:
train

User_id with most questions answered.

In [ ]:
fig = px.histogram(train,
                   x='user_id', 
                   histfunc='count',
                  )
fig.update_layout(showlegend =False,
                 width=1000,
                 height=600)
fig.show()

User_id with longest time stamp

In [ ]:
fig = px.histogram(train,
                   x='user_id', 
                   y='timestamp',
                   histfunc='max',
                  )
fig.update_layout(showlegend =False,
                 width=1000,
                 height=600)
fig.show()

Average of answered_correctly each user

In [ ]:
fig = px.histogram(train,
                   x='user_id', 
                   y='answered_correctly',
                   histfunc='avg',
                  )
fig.update_layout(showlegend =False,
                 width=1000,
                 height=600)
fig.show()

avg of answered correctly each question

In [ ]:
fig = px.histogram(train,
                   x='question_id', 
                   y='answered_correctly',
                   histfunc='avg',
                  )
fig.update_layout(showlegend =False,
                 width=1000,
                 height=600)
fig.show()

avg of answered correctly each part

In [ ]:
fig = px.histogram(train,
                   x='part', 
                   y='answered_correctly',
                   histfunc='avg',
                  )
fig.update_layout(showlegend =False,
                 width=1000,
                 height=600)
fig.show()

Check the correlation with answered_correctly

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='index', y ='answered_correctly', data=train.corr().drop('answered_correctly').reset_index())

all is bad, no one is more than 0.1

For now I will drop row_id, content_type_id, user_answer, correct_answer.<br>
I think those are no corr even in real life.<br>
I mean we don't care correct answer, is it 1 or 2 or 3

In [ ]:
train = train.drop(['row_id', 'content_type_id', 'user_answer', 'correct_answer'], axis=1)

How do you define the question is hard or easy?<br>
Yap!, by count average of how many answered correctly in each question.<br>
So. we gonna groupby question_id and see the average of answered_correctly.

In [ ]:
question_answered_correctly = train.groupby('question_id').mean()['answered_correctly'].reset_index()
question_answered_correctly

We gonna merge them with train data frame and name it as PQAC (percentage of questions answered correctly)

In [ ]:
question_answered_correctly = question_answered_correctly.rename(columns={'answered_correctly':'PQAC'})
train = train.merge(question_answered_correctly)

Next, they already divided question into part. <br>
Let's which question is hard by part.

In [ ]:
part_answered_correctly = train.groupby('part').mean()['answered_correctly'].reset_index()
part_answered_correctly

Again, We merge with our train dataframe. As PPQAC (percentage of part questions answered correctly)

In [ ]:
part_answered_correctly = part_answered_correctly.rename(columns={'answered_correctly':'PPQAC'})
train = train.merge(part_answered_correctly)

How do you define user is smart or not? <br>
Maybe they give many correct answer

In [ ]:
user_answered_correctly = train.groupby('user_id').mean()['answered_correctly'].reset_index()
user_answered_correctly

Again, We merge with our train dataframe. As UAC (percentage of user answered correctly)

In [ ]:
user_answered_correctly = user_answered_correctly.rename(columns={'answered_correctly':'UAC'})
train = train.merge(user_answered_correctly)

how about average user to answer?

In [ ]:
avg_user_answered = (train.groupby('user_id').max()['timestamp']/train.groupby('user_id').count()['timestamp']).reset_index()
avg_user_answered

We merge with our train dataframe. As AUA (Average User Answer)

In [ ]:
avg_user_answered = avg_user_answered.rename(columns={'timestamp':'AUA'})
train = train.merge(avg_user_answered)

But quick to answer question is not enough for people to be called smart. <br>
So I multiple (percentage of user answered correctly and Average User Answer <br>
And call it AUAC (Average User Answered Correctly)

In [ ]:
train['AUAC'] = train['UAC']*train['AUA']

Check again the correlation

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(x='index', y ='answered_correctly', data=train.corr().drop('answered_correctly').reset_index())

In [ ]:
from datetime import datetime

In [ ]:
train['datetime'] = pd.to_datetime(train['timestamp'], unit='ms')

In [ ]:
train.index = train['datetime']

In [ ]:
z = train.groupby('user_id').resample('1D').count()[0:20]['answered_correctly'].reset_index()

In [ ]:
z = z.rename(columns={'answered_correctly':'question/day', 'datetime': 'date_id'})
z

In [ ]:
train = train.reset_index(drop=True)

In [ ]:
train['date_id'] = train['datetime'].dt.date

In [ ]:
train[20:40]

In [ ]:
sns.scatterplot(x='timestamp', y='answered_correctly',data=train)

In [ ]:
fig = px.line(train,
              x='timestamp', 
              y='answered_correctly',
              color='user_id',
              #text='content_id',
                 #symbol='Outlet_Type',
                 #text='Outlet_Identifier',
             )#.update_yaxes(categoryorder='total ascending')

fig.update_traces(marker=dict(size=12,),
                  textposition='top center',
                  textfont=dict(family='Arial',size=12),
              
                 )
fig.update_layout(
    height=600,
)

fig.show()